In [7]:
# ===================== CORE IMPORTS =====================
import os
import cv2
import nibabel as nib
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
# =======================================================

# ===================== GPU SAFETY =====================
tf.keras.mixed_precision.set_global_policy("mixed_float16")
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
# =====================================================

# ---------------- CONFIG ----------------
database_path = r"D:\Projects\BioX_and_WnCC-Brain_Tumor_Segmentation\Brain_Tumor_Segmentation\Week_3&4\archive\BraTS2021_Training_Data"

IMG_SIZE = 128
SLICES_PER_STEP = 16     # 🔥 CRITICAL (NOT 80)
PATIENT_BATCH = 1        # 🔥 MUST BE 1
EPOCHS = 20
DTYPE = np.float32

# ---------------- LOGGING ----------------
def log(msg):
    print(f"[INFO] {msg}")

# ---------------- MEMORY-SAFE SLICE LOADER ----------------
def load_random_slices(path):
    img = nib.load(path)
    data = img.dataobj

    z = data.shape[2]
    center = z // 2
    half = SLICES_PER_STEP // 2
    start = max(0, center - half)
    end = min(z, center + half)

    slices = []
    for i in range(start, end):
        sl = np.asarray(data[:, :, i], dtype=DTYPE)
        sl = cv2.resize(sl, (IMG_SIZE, IMG_SIZE))
        if sl.max() > 0:
            sl /= sl.max()
        slices.append(sl)

    return np.stack(slices, axis=0)

# ---------------- DATA GENERATOR ----------------
class BraTSGenerator(Sequence):
    def __init__(self, patient_dirs, shuffle=True):
        self.patient_dirs = patient_dirs
        self.shuffle = shuffle
        self.indices = np.arange(len(patient_dirs))
        self.on_epoch_end()
        log(f"Generator ready with {len(patient_dirs)} patients")

    def __len__(self):
        return len(self.patient_dirs)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        log("Epoch shuffle done")

    def __getitem__(self, idx):
        patient = self.patient_dirs[self.indices[idx]]
        pid = os.path.basename(patient)

        log(f"Loading patient {pid}")

        flair = os.path.join(patient, f"{pid}_flair.nii.gz")
        seg   = os.path.join(patient, f"{pid}_seg.nii.gz")

        flair_slices = load_random_slices(flair)
        seg_slices   = load_random_slices(seg)

        flair_slices = np.expand_dims(flair_slices, -1)
        flair_slices = np.repeat(flair_slices, 4, axis=-1)

        X = flair_slices
        y = np.expand_dims(seg_slices, -1)

        log(f"✓ Patient {pid} loaded successfully")
        return X, y

# ---------------- PATIENT SPLIT ----------------
patients = sorted([
    os.path.join(database_path, p)
    for p in os.listdir(database_path)
    if p.startswith("BraTS2021_")
])

split = int(0.8 * len(patients))
train_gen = BraTSGenerator(patients[:split])
val_gen   = BraTSGenerator(patients[split:], shuffle=False)

log(f"Train patients: {len(train_gen)} | Val patients: {len(val_gen)}")

# ---------------- METRICS ----------------
def dice_coef(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    return (2. * inter) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-6)

def iou(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - inter
    return inter / (union + 1e-6)

# ---------------- U-NET ----------------
def unet(input_shape):
    inp = layers.Input(input_shape)

    c1 = layers.Conv2D(32, 3, activation="relu", padding="same")(inp)
    c1 = layers.Conv2D(32, 3, activation="relu", padding="same")(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(64, 3, activation="relu", padding="same")(p1)
    c2 = layers.Conv2D(64, 3, activation="relu", padding="same")(c2)
    p2 = layers.MaxPooling2D()(c2)

    c3 = layers.Conv2D(128, 3, activation="relu", padding="same")(p2)
    c3 = layers.Conv2D(128, 3, activation="relu", padding="same")(c3)

    u1 = layers.UpSampling2D()(c3)
    u1 = layers.Concatenate()([u1, c2])
    c4 = layers.Conv2D(64, 3, activation="relu", padding="same")(u1)

    u2 = layers.UpSampling2D()(c4)
    u2 = layers.Concatenate()([u2, c1])
    c5 = layers.Conv2D(32, 3, activation="relu", padding="same")(u2)

    out = layers.Conv2D(1, 1, activation="sigmoid", dtype="float32")(c5)
    return Model(inp, out)

model = unet((IMG_SIZE, IMG_SIZE, 4))
model.compile(
    optimizer=Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[dice_coef, iou]
)

model.summary()

# ---------------- TRAIN ----------------
log("Starting training...")
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    workers=1,
    use_multiprocessing=False
)

model.save("brats2021_unet_model.h5")
log("Training finished successfully — model saved.")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Projects\\BioX_and_WnCC-Brain_Tumor_Segmentation\\Brain_Tumor_Segmentation\\Week_3&4\\archive\\BraTS2021_Training_Data'